<!-- TABS -->
# Retrieval augmented generation

<!-- TABS -->
## Connect to superduper

:::note
Note that this is only relevant if you are running superduper in development mode.
Otherwise refer to "Configuring your production system".
:::

In [1]:
from superduper import superduper

db = superduper('mongomock:///test_db')

2024-Aug-25 20:58:32.27| INFO     | Duncans-MBP.fritz.box| superduper.misc.plugins:13   | Loading plugin: mongodb
2024-Aug-25 20:58:32.37| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:103  | Building Data Layer
2024-Aug-25 20:58:32.37| INFO     | Duncans-MBP.fritz.box| superduper.base.build:171  | Configuration: 
 +---------------+----------------------+
| Configuration |        Value         |
+---------------+----------------------+
|  Data Backend | mongomock:///test_db |
+---------------+----------------------+


<!-- TABS -->
## Get useful sample data

In [2]:
# <tab: Text>
# !curl -O https://superduperdb-public-demo.s3.amazonaws.com/text.json
import json

with open('text.json', 'r') as f:
    data = json.load(f)

In [ ]:
# <tab: PDF>
!curl -O https://superduperdb-public-demo.s3.amazonaws.com/pdfs.zip && unzip -o pdfs.zip
import os

data = [f'pdfs/{x}' for x in os.listdir('./pdfs') if x.endswith('.pdf')]

In [3]:
datas = [{'x': d} for d in data]

<!-- TABS -->
## Insert simple data

After turning on auto_schema, we can directly insert data, and superduper will automatically analyze the data type, and match the construction of the table and datatype.

In [4]:
from superduper import Document

ids = db.execute(db['docs'].insert([Document(data) for data in datas]))

2024-Aug-25 20:58:38.99| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:363  | Table docs does not exist, auto creating...
2024-Aug-25 20:58:38.99| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:369  | Creating table docs with schema {('x', 'str'), ('_fold', 'str')}
2024-Aug-25 20:58:38.99| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:415  | Leaf str already exists
2024-Aug-25 20:58:39.04| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:344  | Inserted 210 documents into docs


<!-- TABS -->
## Apply a chunker for search

:::note
Note that applying a chunker is ***not*** mandatory for search.
If your data is already chunked (e.g. short text snippets or audio) or if you
are searching through something like images, which can't be chunked, then this
won't be necessary.
:::

In [5]:
# <tab: Text>
from superduper import model

CHUNK_SIZE = 200

@model(flatten=True, model_update_kwargs={})
def chunker(text):
    text = text.split()
    chunks = [' '.join(text[i:i + CHUNK_SIZE]) for i in range(0, len(text), CHUNK_SIZE)]
    return chunks

In [ ]:
# <tab: PDF>
!pip install -q "unstructured[pdf]"
from superduper import model
from unstructured.partition.pdf import partition_pdf

CHUNK_SIZE = 500

@model(flatten=True)
def chunker(pdf_file):
    elements = partition_pdf(pdf_file)
    text = '\n'.join([e.text for e in elements])
    chunks = [text[i:i + CHUNK_SIZE] for i in range(0, len(text), CHUNK_SIZE)]
    return chunks

Now we apply this chunker to the data by wrapping the chunker in `Listener`:

In [6]:
from superduper import Listener

upstream_listener = Listener(
    model=chunker,
    select=db['docs'].select(),
    key='x',
    uuid="chunker",
    identifier='chunker',
)

## Select outputs of upstream listener

:::note
This is useful if you have performed a first step, such as pre-computing 
features, or chunking your data. You can use this query to 
operate on those outputs.
:::

<!-- TABS -->
## Build text embedding model

In [7]:
# <tab: OpenAI>
import os
os.environ['OPENAI_API_KEY'] = 'sk-<secret>'
from superduper_openai import OpenAIEmbedding

embedding_model = OpenAIEmbedding(identifier='text-embedding-ada-002')

In [ ]:
# <tab: JinaAI>
import os
from superduper_jina import JinaEmbedding

os.environ["JINA_API_KEY"] = "jina_xxxx"
 
# define the model
embedding_model = JinaEmbedding(identifier='jina-embeddings-v2-base-en')

In [ ]:
# <tab: Sentence-Transformers>
!pip install sentence-transformers
from superduper import vector
import sentence_transformers
from superduper_sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer(
    identifier="embedding",
    object=sentence_transformers.SentenceTransformer("BAAI/bge-small-en"),
    datatype=vector(shape=(1024,)),
    postprocess=lambda x: x.tolist(),
    predict_kwargs={"show_progress_bar": True},
)

## Create vector-index

In [8]:
from superduper import VectorIndex, Listener

vector_index_name = 'vector-index'

vector_index = \
    VectorIndex(
        vector_index_name,
        indexing_listener=Listener(
            key=upstream_listener.outputs,      # the `Document` key `model` should ingest to create embedding
            select=db[upstream_listener.outputs].select(),       # a `Select` query telling which data to search over
            model=embedding_model,         # a `_Predictor` how to convert data to embeddings
            uuid="embedding-listener",
            identifier='embedding-listener',
            upstream=[upstream_listener],
        )
    )

<!-- TABS -->
## Create Vector Search Model

In [10]:
item = {'_outputs__chunker': '<var:query>'}

In [11]:
from superduper.components.model import QueryModel

vector_search_model = QueryModel(
    identifier="VectorSearch",
    select=db[upstream_listener.outputs].like(item, vector_index=vector_index_name, n=5).select(),
    # The _source is the identifier of the upstream data, which can be used to locate the data from upstream sources using `_source`.
    postprocess=lambda docs: [{"text": doc['_outputs__chunker'], "_source": doc["_source"]} for doc in docs],
    db=db
)

<!-- TABS -->
## Build LLM

In [12]:
# <tab: OpenAI>
from superduper_openai import OpenAIChatCompletion

llm = OpenAIChatCompletion(identifier='llm', model='gpt-3.5-turbo')

In [ ]:
# <tab: Anthropic>
from superduper_anthropic import AnthropicCompletions
import os

os.environ["ANTHROPIC_API_KEY"] = "sk-xxx"

predict_kwargs = {
    "max_tokens": 1024,
    "temperature": 0.8,
}

llm = AnthropicCompletions(identifier='llm', model='claude-2.1', predict_kwargs=predict_kwargs)

In [ ]:
# <tab: vLLM>
from superduper_vllm import VllmModel

predict_kwargs = {
    "max_tokens": 1024,
    "temperature": 0.8,
}


llm = VllmModel(
    identifier="llm",
    model_name="TheBloke/Mistral-7B-Instruct-v0.2-AWQ",
    vllm_kwargs={
        "gpu_memory_utilization": 0.7,
        "max_model_len": 1024,
        "quantization": "awq",
    },
    predict_kwargs=predict_kwargs,
)

In [ ]:
# <tab: Transformers>
from superduper_transformers import LLM

llm = LLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", load_in_8bit=True, device_map="cuda", identifier="llm", predict_kwargs=dict(max_new_tokens=128))

In [ ]:
# <tab: Llama.cpp>
!huggingface-cli download TheBloke/Mistral-7B-Instruct-v0.2-GGUF mistral-7b-instruct-v0.2.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False

from superduper_llama_cpp.model import LlamaCpp
llm = LlamaCpp(identifier="llm", model_name_or_path="mistral-7b-instruct-v0.2.Q4_K_M.gguf")

## Answer question with LLM

In [13]:
from superduper import model
from superduper.components.graph import Graph, input_node

prompt_template = (
    "Use the following context snippets, these snippets are not ordered!, Answer the question based on this context.\n"
    "{context}\n\n"
    "Here's the question: {query}"
)

@model
def build_prompt(query, docs):
    chunks = [doc["text"] for doc in docs]
    context = "\n\n".join(chunks)
    prompt = prompt_template.format(context=context, query=query)
    return prompt

# We build a graph to handle the entire pipeline

# create a input node, only have one input parameter `query`
in_ = input_node('query')
# pass the query to the vector search model
vector_search_results = vector_search_model(query=in_)
# pass the query and the search results to the prompt builder
prompt = build_prompt(query=in_, docs=vector_search_results)
# pass the prompt to the llm model
answer = llm(prompt)
# create a graph, and the graph output is the answer
rag = answer.to_graph("rag")

By applying the RAG model to the database, it will subsequently be accessible for use in other services.

In [16]:
from superduper import Application

app = Application(
    'rag-app',
    components=[
        upstream_listener,
        vector_index,
        vector_search_model,
        rag,
    ]
)

db.apply(app)

2024-Aug-25 21:00:42.45| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:415  | Leaf _input already exists
2024-Aug-25 21:00:42.45| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:415  | Leaf llm already exists


([],
 Application(identifier='rag-app', uuid='e1e98a1c9ef244d1b3386c78bc74d391', upstream=None, plugins=None, cache=False, components=[Listener(identifier='chunker', uuid='chunker', upstream=None, plugins=None, cache=False, key='x', model=ObjectModel(identifier='chunker', uuid='3bc1b7c65f8f493d97344256718517ae', upstream=None, plugins=None, cache=False, signature='*args,**kwargs', datatype=None, output_schema=None, flatten=True, model_update_kwargs={}, predict_kwargs={}, compute_kwargs={}, validation=None, metric_values={}, num_workers=0, object=<function chunker at 0x150180040>), select=docs.select(), predict_kwargs={}, predict_id='chunker'), VectorIndex(identifier='vector-index', uuid='6e34deef08d54c66acf35b58f8a5fa0c', upstream=None, plugins=None, cache=False, indexing_listener=Listener(identifier='embedding-listener', uuid='embedding-listener', upstream=[Listener(identifier='chunker', uuid='chunker', upstream=None, plugins=None, cache=False, key='x', model=ObjectModel(identifier='c

You can now load the model elsewhere and make predictions using the following command.

In [15]:
rag = db.load("model", 'rag')
print(rag.predict("Tell me about superduper")[0])

2024-Aug-25 20:59:50.29| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:889  | {}
SuperDuperDB is a virtual AI-datalayer open-sourced in Python under the Apache 2.0 license. It allows developers to connect an AI development environment directly to data, connect an AI production environment directly to data, and create a flexible platform connecting AI and data for collaboration. SuperDuperDB can handle classical AI/machine learning paradigms like classification, regression, forecasting, clustering, and more. Users can choose how to deploy SuperDuperDB, use their own models or integrate AI APIs and frameworks, work with various data types, version and track functionality, and control data exposure to API services. Key features include AI integration with existing data infrastructure, streaming inference, scalable model training, model chaining, and a simple but extendable interface. Users can get started with SuperDuperDB by installing and configuring it, trying code snippe

## Create template

In [17]:
from superduper import Template

template = Template('rag-template', template=app, substitutions={'docs': 'collection'})

2024-Aug-25 21:00:59.65| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:415  | Leaf chunker already exists
2024-Aug-25 21:00:59.65| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:415  | Leaf chunker already exists
2024-Aug-25 21:00:59.66| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:415  | Leaf dill_lazy already exists
2024-Aug-25 21:00:59.66| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:415  | Leaf docs-select already exists
2024-Aug-25 21:00:59.66| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:415  | Leaf VectorSearch already exists
2024-Aug-25 21:00:59.66| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:415  | Leaf dill already exists
2024-Aug-25 21:00:59.66| WARNING  | Duncans-MBP.fritz.box| superduper.base.document:415  | Leaf -outputs-chunker-like-outputs-chunker-<var:query>-vector-index-vector-index-n-5-select already exists
2024-Aug-25 21:00:59.66| WARNING  | Duncans-MBP.fritz.box| superduper.base.docum

In [19]:
template.export('.')